In [4]:
from pyannote.audio import Pipeline
from transformers import pipeline
from indicnlp.normalize.indic_normalize import IndicNormalizerFactory
import librosa
import os
from indicnlp.normalize.indic_normalize import DevanagariNormalizer

In [5]:
hf_token = os.environ.get('HF_TOKEN')

In [ ]:
diarization_pipeline = Pipeline.from_pretrained(
    "pyannote/speaker-diarization-3.1",
    use_auth_token=hf_token )
audio_file= "outputs/videoplayback.wav"
diarization = diarization_pipeline(audio_file)

d:\final karya\myenv\Lib\site-packages\pyannote\audio\models\blocks\pooling.py:104: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\ReduceOps.cpp:1831.)
  std = sequences.std(dim=-1, correction=1)


In [8]:
whisper_asr = pipeline(
    "automatic-speech-recognition",
    model="openai/whisper-large-v2",
)
indic_factory = IndicNormalizerFactory()
normalizer = DevanagariNormalizer(
    lang='hi',
    remove_nuktas=False,
    nasals_mode='do_nothing',
    do_normalize_chandras=False,
    do_normalize_vowel_ending=False
)

Device set to use cpu


In [36]:
transcript = []
for segment, _, speaker in diarization.itertracks(yield_label=True):
    audio, sr = librosa.load(
        "/content/videoplayback.wav",
        sr=16000,
        offset=segment.start,
        duration=segment.duration,
        mono=True
    )

    try:
        result = whisper_asr(
            audio,
            return_timestamps=True  # Pipeline-specific param
        )
        transcript.append({
            "speaker": speaker,
            "start": float(segment.start),
            "end": float(segment.end),
            "text": result["text"]
        })
    except Exception as e:
        print(f"Failed segment {segment.start}-{segment.end}: {str(e)}")

ImportError: Numba needs NumPy 2.1 or less. Got NumPy 2.2.

In [ ]:
transcript